# Assistente de Correção Ortográfica e Gramatical da Língua Portuguesa

## Instalando bibliotecas e dependências

In [ ]:
!wget https://languagetool.org/download/LanguageTool-stable.zip
!unzip LanguageTool-stable.zip

In [ ]:
!wget -O jre-8u251-linux-x64.tar.gz 'https://javadl.oracle.com/webapps/download/AutoDL?xd_co_f=M2FmMjIzNjktMjBjNC00Y2QxLWFjNGYtMDI5MWMzNDM1MTFh&BundleId=242050_3d5a2bb8f8d4428bbe94aed7ec7ae784'

In [ ]:
!tar -xzvf jre-8u251-linux-x64.tar.gz

## Carregando serviço de correção ortográfica na porta local 7001

In [ ]:
!nohup jre1.8.0_251/bin/java -cp LanguageTool-5.0/languagetool-server.jar org.languagetool.server.HTTPServer --port 7001 --allow-origin "*" > corretor.run.log &

In [ ]:
!cat corretor.run.log

In [ ]:
# testando
!curl --data "language=pt-BR&text=A menino chutou o bola." http://localhost:7001/v2/check

## Método para Correção Ortográfica

In [ ]:
import json
import urllib
from urllib.request import urlopen

def corretor_ortografico(text):

  res = []

  try:
    response = urlopen("http://localhost:7001/v2/check?language=pt-BR&text="+urllib.parse.quote_plus(text))
    corto_json = json.loads(response.read().decode())
    return corto_json['matches']
    
  except:
    return [-1] # some error
  
  return res


text = 'O menina chutou a bola.'
corretor_ortografico(text)

# Assistente de Detecção de Plágio

## Instalando bibliotecas e dependências

In [ ]:
!git clone https://github.com/attardi/wikiextractor.git

In [ ]:
!pip install elasticsearch-loader

## Compilando Base de Referência para Verificação de Plágio: Wikipedia - PT-BR

In [ ]:
!wget https://dumps.wikimedia.org/ptwiki/latest/ptwiki-latest-pages-articles.xml.bz2

In [ ]:
!python3 wikiextractor/WikiExtractor.py ptwiki-latest-pages-articles.xml.bz2 --processes 8 --json -o WIKIJSON

## Indexando base de referência em um sistema de recuperação de informação (Elasticsearch Porta Local 9200)


In [ ]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0
# start server
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

In [ ]:
# testando (só continuar depois de verificar que elasticsearch esta executando)
!curl -X GET "localhost:9200/"

In [ ]:
# indexando
!elasticsearch_loader  --index plagio --type wikipediaptbr json --json-lines WIKIJSON/AA/wiki_00
!elasticsearch_loader  --index plagio --type wikipediaptbr json --json-lines WIKIJSON/AA/wiki_01
!elasticsearch_loader  --index plagio --type wikipediaptbr json --json-lines WIKIJSON/AA/wiki_02

In [ ]:
!wget -O teste_plagio.json "http://localhost:9200/_search?q=manchas+solares"
!cat teste_plagio.json

## Método para Plágio Scoring

In [ ]:
import json
import urllib
from urllib.request import urlopen

def plagio_detector(query):

  res = []

  try:
    response = urlopen("http://localhost:9200/_search/?q="+urllib.parse.quote_plus(query))
    plagio_json = json.loads(response.read().decode())

    for hits in plagio_json['hits']['hits']:
      score = hits['_score']/30.0
      if score > 1.0: score = 1.0
      res.append( [score, hits['_source']] )
  except:
    return [-1] # some error
  
  return res


query = 'Astronomia é uma ciência natural que estuda corpos celestes'
plagio_detector(query)

# Assistente de Fuga de Tema (Questões Aberts / Discursivas)

## Instalando bibliotecas e dependências

In [ ]:
!pip install -U sentence-transformers



In [ ]:
!pip install transformers==2.11.0

## Algoritmo para detecção de fuga de tema usando modelo de linguagem (DistilBert multilíngua)

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

# Load Sentence model (based on BERT) from URL
from sentence_transformers import SentenceTransformer, LoggingHandler
import numpy as np
import logging

#### Just some code to print debug information to stdout
np.set_printoptions(threshold=100)

logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

language_model = SentenceTransformer('distiluse-base-multilingual-cased')


def fuga_score(a,b):
  cos_sim = dot(a, b)/(norm(a)*norm(b))
  if cos_sim < 0: cos_sim =0.0
  score = 1.0 - cos_sim
  return score

def fuga_tema(texto_entrada, texto_referencia):
  global language_model

  if len(texto_entrada) < 50: return -1
  if len(texto_referencia) < 50: return -1

  a = language_model.encode([texto_entrada])[0]
  b = language_model.encode([texto_referencia])[0]
  return fuga_score(a,b)



In [ ]:
# testando fuga de tema (o quanto o texto de entrada se diferencia semanticamente de um texto de referencia)
entrada = 'Paraná chega a 20mil casos de coronavírus e 586 mortes'
referencia = 'Paraná ultrapassa a marca de 20 mil casos confirmados de coronavírus'
fuga_tema(entrada,referencia)

# API de Integração dos Assistentes de Provas Virtuais 3.0 (questões abertas/discursivas)

In [ ]:
from flask import Flask,request
from flask import jsonify


app = Flask(__name__)

@app.route("/")
def service():
    try:
        service = request.args.get('service')
        print('service: ',service)

        if service == 'corretor':
          text = request.args.get('text')
          print('text: ',text)
          res = corretor_ortografico(text)
          return jsonify(res),200

        if service == 'plagio':
          text = request.args.get('text')
          print('text: ',text)
          res = plagio_detector(text)
          return jsonify(res),200

        if service == 'fuga_tema':
          text = request.args.get('text')
          reference = request.args.get('reference')
          print('text: ',text)
          print('reference: ',reference)
          res = fuga_tema(text,reference)
          return jsonify([res]),200

        return jsonify([-1,'No service defined?']),200

    except Exception as e:
        print(e)
        print("An exception occurred")
        return str([-1]),200

if __name__ == '__main__':
    app.run()

## Executar apenas para debug em modo Google Colab

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

# Open a HTTP tunnel on the default port 80
public_url = ngrok.connect(port=5000)
print(public_url)